In [ ]:
from pathlib import Path

llm_model_id = "microsoft/Phi-3.5-mini-instruct"
llm_model_path = "../model/phi-3.5-mini-instruct-ov"

if not Path(llm_model_path).exists():
    !optimum-cli export openvino --model {llm_model_id} --task text-generation-with-past --weight-format int4 --group-size 128 --ratio 0.6  --sym  --trust-remote-code {llm_model_path}

In [ ]:
from transformers import AutoConfig, AutoTokenizer
from optimum.intel.openvino import OVModelForCausalLM

In [ ]:
ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}

In [ ]:
ov_model = OVModelForCausalLM.from_pretrained(
    llm_model_path,
    device='GPU',
    ov_config=ov_config,
    config=AutoConfig.from_pretrained(llm_model_path, trust_remote_code=True),
    trust_remote_code=True,
)

In [ ]:
tok = AutoTokenizer.from_pretrained(llm_model_path, trust_remote_code=True)

In [ ]:
tokenizer_kwargs =  {"add_special_tokens": False}

In [ ]:
prompt = "<|user|>\n你了解 .NET 吗?\n<|end|><|assistant|>\n"
input_tokens = tok(prompt, return_tensors="pt", **tokenizer_kwargs)

In [ ]:
answer = ov_model.generate(**input_tokens, max_new_tokens=1024)

In [ ]:
tok.batch_decode(answer, skip_special_tokens=True)[0]


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan hasil yang akurat, harap diingat bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
